In [ ]:
[{$unwind: {
  path: "$sessions"
}}, {$unwind: {
  path: "$sessions.actions"
}}, {$group: {
  "_id": {
    "number": "$number",
    "type": "$sessions.actions.type"
  },
  "created_at": {
    "$push": "$sessions.actions.created_at"
  },
  "count": {
    "$sum": 1
  }
}}, {$group: {
  "_id": {
    "number": "$_id.number"
  },
  "actions": {
    "$push": {
      "type": "$_id.type",
      "last": {
        "$max": "$created_at"
      },
      "count": "$count"
    }
  }
}}, {$set: {
  "actions": {
    "$function": {
      "body": "function(actions) {try {var act = ['create', 'read', 'update', 'delete']; act.forEach((entry) => {if (actions.filter(el => el.type === entry).length == 0) {actions.push({'count': 0, type: entry, last: null });}})} catch(e){} return actions}",
      "args": ["$actions"],
      "lang": "js"
    }
  }
}}, {$project: {
  _id: 0,
  number: "$_id.number",
  actions: 1
}}]

In [27]:
import sqlite3 as sql
from datetime import datetime
from dateutil import parser
import pandas as pd

def make_db():
    conn = sql.connect("task_2.db")
    cur = conn.cursor()

    cur.execute("""CREATE TABLE IF NOT EXISTS accrual(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date INT,
        month INT)
    """)

    cur.execute("""CREATE TABLE IF NOT EXISTS payment(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date INT,
        month INT)
    """)
    add_accrual(accruals)
    add_payments(payments)


def get_accruals():
    con = sql.connect("task_2.db")
    cur = con.cursor()
    cur.execute("SELECT * FROM accrual ORDER BY date DESC")
    accruals = cur.fetchall()
    con.close()
    return accruals


def get_payments():
    con = sql.connect("task_2.db")
    cur = con.cursor()
    cur.execute("SELECT * FROM payment ORDER BY date ASC")
    payments = cur.fetchall()
    con.close()
    return payments


def add_accrual(accruals):
    con = sql.connect("task_2.db")
    cur = con.cursor()
    cur.executemany("INSERT INTO accrual (date, month) VALUES(?, ?);", accruals)
    con.commit()
    con.close()


def add_payments(payments):
    con = sql.connect("task_2.db")
    cur = con.cursor()
    cur.executemany("INSERT INTO payment (date, month) VALUES(?, ?);", payments)
    con.commit()
    con.close()


accruals = [
    ('2021-08-31', 8),
    ('2021-07-25', 7),
    ('2021-07-18', 7),
    ('2021-07-22', 7),
    ('2021-06-16', 6),
    ('2021-05-18', 5),
    ('2021-03-23', 3),
    ('2021-07-07', 7)
]

payments = [
    ('2021-09-11', 9),
    ('2021-07-25', 7),
    ('2021-07-29', 7),
    ('2021-06-15', 6),
    ('2021-06-18', 6),
    ('2021-07-13', 7),
    ('2021-02-14', 2)
]


def payment_mapping(payments, accruals):
    not_matched = payments[:]
    matches = []
    
    for payment in payments:
        payment_date = parser.parse(payment[1])
        payment_month = payment[2]

        for accrual in accruals:
            accrual_date = parser.parse(accrual[1])
            accrual_month = accrual[2]

            if accrual_month == payment_month and payment_date > accrual_date: # Будем брать ближайший к дате платежа долг, если есть несколько долгов в месяце
                matches.append((accrual, payment))
                not_matched.remove(payment)
                accruals.remove(accrual)
                break
            elif payment_date > accrual_date:
                oldest_accrual = accruals[-1]
                matches.append((oldest_accrual, payment))
                not_matched.remove(payment)
                accruals.remove(oldest_accrual)
                break

    return matches, not_matched
        

if __name__ == '__main__':
    make_db()
    conn = sql.connect("task_2.db")
    cur = conn.cursor()

    accruals = get_accruals()
    payments = get_payments()
    # print("Payments:\n", payments)
    # print("Accruals:\n", accruals)

    matches, not_matched = payment_mapping(payments, accruals)
    print("Matches:")
    print(pd.DataFrame(matches, columns =['Accrual', 'Payment']))
    print("Not matched payments:\n", *not_matched)


Matches:
               Accrual              Payment
0  (79, 2021-03-23, 3)  (67, 2021-06-15, 6)
1  (77, 2021-06-16, 6)  (68, 2021-06-18, 6)
2  (80, 2021-07-07, 7)  (69, 2021-07-13, 7)
3  (76, 2021-07-22, 7)  (65, 2021-07-25, 7)
4  (74, 2021-07-25, 7)  (66, 2021-07-29, 7)
5  (78, 2021-05-18, 5)  (64, 2021-09-11, 9)
Not matched payments:
 (70, '2021-02-14', 2)


In [21]:
sql = 'DELETE  FROM payment'
cur = conn.cursor()
cur.execute(sql)
conn.commit()
sql = 'DELETE  FROM accrual'
cur = conn.cursor()
cur.execute(sql)
conn.commit()